In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import os,re
import numpy as np
import pandas as pd
%matplotlib inline

/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
test_data = pd.read_csv("test_final.csv",encoding = "ISO-8859-1", header = None)
print(len(test_data))
test_data.head(10)

1701


,0
0,Couldn't be more proud of @HillaryClinton. Her...
1,This election is too important to sit out. Go ...
2,"Once again, we will have a government of, by a..."
3,"On National #VoterRegistrationDay, make sure y..."
4,Great afternoon in Little Havana with Hispanic...
5,It's #NationalVoterRegistrationDay. Celebrate ...
6,"""I love this country.\rIÛªm proud of this cou..."
7,"""What kind of a person would want to root for ..."
8,"""I donÛªt think that any family should have t..."
9,Join Hillary live in NC for her first rally si...


In [3]:
train_data = pd.read_csv("train.csv",header = None)
print(len(train_data))
train_data.describe()
train_data[0].head(10)

4743


0    The question in this election: Who can put the...
1    Last night, Donald Trump said not paying taxes...
2    If we stand together, there's nothing we can't...
3    Both candidates were asked about how they'd co...
4    Join me for a 3pm rally - tomorrow at the Mid-...
5    When Donald Trump goes low...register to vote:...
6    3) Has Trump offered a single proposal to redu...
7    The election is just weeks away. Check if you'...
8    Hillary Clinton's Campaign Continues To Make F...
9    'CNBC, Time magazine online polls say Donald T...
Name: 0, dtype: object

In [4]:
labels_train = pd.read_csv("labels_train_tweets.csv",header = None)
labels_train[0] = labels_train[0].map({'HC':1,'DT':0})
#Hillary = 1 , DT = 0

In [5]:
labels_train.head(10)

,0
0,1
1,1
2,1
3,1
4,0
5,1
6,1
7,1
8,0
9,0


In [6]:
from keras import backend as K
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [7]:
#The actual vocab size was around 12k, but using a larger number 
#reduces the probability of collisions from the hash function.
vocab_size = 20000
encoded_tweets = [one_hot(d, vocab_size) for d in train_data[0]]
encoded_test = [one_hot(d, vocab_size) for d in test_data[0]]
tweet_lengths = [len(t) for t in encoded_tweets]
test_lengths = [len(t) for t in encoded_test]
max_length = 0
for t in encoded_tweets:
    if len(t) > max_length:
        max_length = len(t)
        
for t in encoded_test:
    if len(t) > max_length:
        max_length = len(t)

labels_train
padded_tweets
tweet_lengths

In [8]:
class SimpleDataIterator():
    def __init__(self, df):
        self.df = df
        self.size = len(self.df)
        self.epochs = 0
        self.shuffle()

    def shuffle(self):
        self.df = self.df.sample(frac=1).reset_index(drop=True)
        self.cursor = 0

    def next_batch(self, n):
        if self.cursor+n-1 > self.size:
            self.epochs += 1
            self.shuffle()
        res = self.df.ix[self.cursor:self.cursor+n-1]
        self.cursor += n
        return res['data'], res['labels'], res['length']

In [9]:
train_dic={}
train_dic["data"] = encoded_tweets
train_dic["labels"] = labels_train[0].ravel().tolist()
train_dic["length"] = tweet_lengths
train_len = len(train_data)
test_len = len(test_data)

train = pd.DataFrame.from_dict(data=train_dic, orient='columns', dtype=None)


test_dic={}
test_dic["data"] = encoded_test
test_dic["length"] = test_lengths
test = pd.DataFrame.from_dict(data=test_dic, orient='columns', dtype=None)

test_input = test.values

data = SimpleDataIterator(train)
d = data.next_batch(3)
print('Input sequences\n', d[0], end='\n\n')
print('Target values\n', d[1], end='\n\n')
print('Sequence lengths\n', d[2])

Input sequences
 0    [12658, 2137, 5316, 3923, 14360, 15733, 7636, ...
1    [14242, 10537, 5928, 15282, 19464, 15733, 3657...
2    [11156, 1197, 13653, 1561, 11559, 11511, 2491,...
Name: data, dtype: object

Target values
 0    0
1    0
2    0
Name: labels, dtype: int64

Sequence lengths
 0    24
1    17
2    23
Name: length, dtype: int64


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


In [10]:
class PaddedDataIterator(SimpleDataIterator):
    def next_batch(self, n):
        if self.cursor+n > self.size:
            self.epochs += 1
            self.shuffle()
        res = self.df.ix[self.cursor:self.cursor+n-1]
        self.cursor += n

        # Pad sequences with 0s so they are all the same length
        maxlen = max(res['length'])
        x = np.zeros([n, maxlen], dtype=np.int32)
        for i, x_i in enumerate(x):
            x_i[:res['length'].values[i]] = res['data'].values[i]

        return x, res['labels'], res['length']

In [11]:
data = PaddedDataIterator(train)
train_data = PaddedDataIterator(test)
d = data.next_batch(3)
print('Input sequences\n', d[0], end='\n\n')

Input sequences
 [[ 2137  3206 15282 12343  1448  5309  1448 10726  3133 12863 13231 11511
   3037    59 17799     0     0     0     0     0     0     0]
 [11185 17878 14315  4154 10726  2882  2338 15282 10636 15733 10006   772
   5754 14360 15282  6206 12712  1414  1491  5776 18197   152]
 [ 5684 15099 18191 16208  5903  9083 16697 15733  7045 13794  2714 16527
  12295 12702 16697 17807  1491  5776 18197 14475     0     0]]



/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [12]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

def build_graph(
    vocab_size = vocab_size,
    state_size = 64,
    batch_size = 189,
    num_classes = 2):

    reset_graph()

    # Placeholders
    x = tf.placeholder(tf.int32, [batch_size, None]) # [batch_size, num_steps]
    seqlen = tf.placeholder(tf.int32, [batch_size])
    y = tf.placeholder(tf.int32, [batch_size])
    keep_prob = tf.placeholder_with_default(1.0, [])

    # Embedding layer
    embeddings = tf.get_variable('embedding_matrix', [vocab_size, state_size])
    rnn_inputs = tf.nn.embedding_lookup(embeddings, x)

    # RNN
    cell = tf.nn.rnn_cell.GRUCell(state_size)
    init_state = tf.get_variable('init_state', [1, state_size],
                                 initializer=tf.constant_initializer(0.0))
    init_state = tf.tile(init_state, [batch_size, 1])
    rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, sequence_length=seqlen,
                                                 initial_state=init_state)

    # Add dropout, as the model otherwise quickly overfits
    rnn_outputs = tf.nn.dropout(rnn_outputs, keep_prob)

    """
    Obtain the last relevant output. The best approach in the future will be to use:

        last_rnn_output = tf.gather_nd(rnn_outputs, tf.pack([tf.range(batch_size), seqlen-1], axis=1))

    which is the Tensorflow equivalent of numpy's rnn_outputs[range(30), seqlen-1, :], but the
    gradient for this op has not been implemented as of this writing.

    The below solution works, but throws a UserWarning re: the gradient.
    """
    idx = tf.range(batch_size)*tf.shape(rnn_outputs)[1] + (seqlen - 1)
    last_rnn_output = tf.gather(tf.reshape(rnn_outputs, [-1, state_size]), idx)

    # Softmax layer
    with tf.variable_scope('softmax'):
        W = tf.get_variable('W', [state_size, num_classes])
        b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
    logits = tf.matmul(last_rnn_output, W) + b
    preds = tf.nn.softmax(logits)
    correct = tf.equal(tf.cast(tf.argmax(preds,1),tf.int32), y)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels = y))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

    return {
        'x': x,
        'seqlen': seqlen,
        'y': y,
        'dropout': keep_prob,
        'loss': loss,
        'ts': train_step,
        'preds': preds,
        'accuracy': accuracy
    }

def train_graph(graph, sess, batch_size = 189, num_epochs = 13, iterator = PaddedDataIterator):
    
    
    sess.run(tf.global_variables_initializer())
    tr = iterator(train)
    te = iterator(test)

    step, accuracy = 0, 0
    tr_losses, te_losses = [], []
    current_epoch = 0
    while current_epoch < num_epochs:
        step += 1
        batch = tr.next_batch(batch_size)
        feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2], g['dropout']: 0.6}
        accuracy_, _ = sess.run([g['accuracy'], g['ts']], feed_dict=feed)
        accuracy += accuracy_

        if tr.epochs > current_epoch:
            current_epoch += 1
            tr_losses.append(accuracy / step)
            step, accuracy = 0, 0

            #eval test set
            """ te_epoch = te.epochs
            while te.epochs == te_epoch:
                step += 1
                batch = te.next_batch(batch_size)
                feed = {g['x']: batch[0], g['y']: batch[1], g['seqlen']: batch[2]}
                accuracy_ = sess.run([g['accuracy'],g['loss']], feed_dict=feed)[0]
                accuracy += accuracy_

            te_losses.append(accuracy / step)"""
            step, accuracy = 0,0
            print("Accuracy after epoch", current_epoch, " - tr:", tr_losses[-1])#, "- te:", te_losses[-1])
    return tr_losses#, te_losses

In [13]:
g = build_graph()
#tr_losses, te_losses = train_graph(g)
sess = tf.Session()
tr_losses = train_graph(g, sess)

/usr/local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


Accuracy after epoch 1  - tr: 0.569800576338
Accuracy after epoch 2  - tr: 0.689523806572
Accuracy after epoch 3  - tr: 0.754708986282
Accuracy after epoch 4  - tr: 0.828783071041
Accuracy after epoch 5  - tr: 0.872380964756
Accuracy after epoch 6  - tr: 0.903068795204
Accuracy after epoch 7  - tr: 0.913227515221
Accuracy after epoch 8  - tr: 0.926984131336
Accuracy after epoch 9  - tr: 0.946031751633
Accuracy after epoch 10  - tr: 0.959365079403
Accuracy after epoch 11  - tr: 0.96931217432
Accuracy after epoch 12  - tr: 0.97417989254
Accuracy after epoch 13  - tr: 0.981587300301


In [22]:
test_input = test
test_input['data'] = test['data'].apply(lambda x : np.array(x))
test_input['length'] = test_input['length'].apply(lambda x : np.array(x))
maxlen = max(test_input['length'])
x = np.zeros([len(test_input), maxlen], dtype=np.int32)
for i, x_i in enumerate(x):
    x_i[:test_input['length'].values[i]] = test_input['data'].values[i]

results = []
import math
for i in range(math.floor(len(x)/189)):
    data_bit = x[i*189:(i+1)*189]
    len_bit = test_input['length'][i*189:(i+1)*189]
    results.append(sess.run([g["preds"]], feed_dict={g['x']: data_bit, g['seqlen'] : len_bit}))
print(results[0])


[array([[  3.06655624e-04,   9.99693394e-01],
       [  5.98663057e-04,   9.99401331e-01],
       [  9.98819172e-01,   1.18075195e-03],
       [  4.18661982e-02,   9.58133817e-01],
       [  9.95761812e-01,   4.23823949e-03],
       [  4.49914439e-03,   9.95500863e-01],
       [  1.96282417e-01,   8.03717613e-01],
       [  1.43793484e-04,   9.99856234e-01],
       [  5.25198237e-04,   9.99474823e-01],
       [  5.71503460e-01,   4.28496510e-01],
       [  4.86638486e-01,   5.13361514e-01],
       [  2.62235343e-01,   7.37764597e-01],
       [  1.78329775e-03,   9.98216689e-01],
       [  5.72474487e-02,   9.42752481e-01],
       [  7.48773575e-01,   2.51226336e-01],
       [  7.62694597e-01,   2.37305343e-01],
       [  6.96491301e-01,   3.03508699e-01],
       [  9.98696983e-01,   1.30300852e-03],
       [  9.63049307e-02,   9.03695047e-01],
       [  9.65713799e-01,   3.42861861e-02],
       [  9.99804676e-01,   1.95357963e-04],
       [  2.37465114e-03,   9.97625411e-01],
       [ 

In [23]:
print(results[0][0][32])


[ 0.0391314   0.96086866]


In [27]:

with open("output.txt", "w") as f:
    f.write("id,realDonaldTrump,HillaryClinton\n")
    #batch of 189
    i = 0
    print (len(results))
    for n in range(len(results)):
        print (len(results[n][0]))
        for y in range(len(results[n][0])):
            f.write("{},{:06f},{:06f}\n".format(i, results[n][0][y][0],results[n][0][y][1]))
            i = i+1
    

9
189
189
189
189
189
189
189
189
189
